In [1]:
import numpy as np

In [2]:
class FFNN:
    def __init__(
            self,
            n_in,
            hidden_nodes: list,
            n_out: int = 1,
            use_bias: bool = True):
        '''Params
        - hidden_nodes: list with n nodes per layer'''
        self.n_in = n_in
        self.hidden_nodes = np.array(hidden_nodes)
        self.use_bias = use_bias
        self.n_out = n_out
        self.Ws = self._init_weights()
        
    def _init_weights(self):
        n_in = self.n_in
        if self.use_bias:
            n_in += 1
        W1 = np.random.normal(size=(n_in, self.hidden_nodes[0]))
        Ws = [W1]
        if len(self.hidden_nodes) > 1:
            for i, n_nodes in enumerate(self.hidden_nodes[1:], 1):
                n_in = self.hidden_nodes[i - 1]
                if self.use_bias:
                    n_in += 1
                W = np.random.normal(size=(n_in, n_nodes))
                Ws.append(W)
        # output
        n_in = self.hidden_nodes[-1]
        if self.use_bias:
            n_in += 1
        W = np.random.normal(size=(n_in, self.n_out))
        Ws.append(W)
        return Ws
        
    def forward_pass(self, X, activation, activation_out):
        activation = {'relu': self._relu}[activation]
        activation_out = {'sigmoid': self._sigmoid}[activation_out]
        X = np.array(X)
        if len(X.shape) == 1:
            X = np.array([X])
        for W in self.Ws[:-1]:
            if self.use_bias:
                X = self._append_bias(X)
            X = X @ W
            X = activation(X)
        # output layer
        if self.use_bias:
            X = self._append_bias(X)
        X = X @ self.Ws[-1]
        X = activation_out(X)
        return X
            
    @staticmethod
    def _append_bias(X):
        bias = np.ones((X.shape[0], 1))
        X = np.concatenate([bias, X], axis=1)
        return X
    
    @staticmethod
    def _relu(X):
        return(X.clip(0, None))
    
    @staticmethod
    def _sigmoid(X):
        return 1 / (1 + np.exp(-X))
    
    def mutate(self, scale: float):
        '''Randomly mutate weights
        Weights will be multiplied by ~N(1, scale)
        In early iterations, it can be large (say 0.5), but should
        gradually diminish as models start to settle
        Parameters:
        - scale: standard deviations
        '''
        for i, W in enumerate(self.Ws):
            noise = np.random.normal(loc=1, scale=scale, size=W.shape)
            W = np.multiply(W, noise)
            self.Ws[i] = W

In [20]:
nn = FFNN(3, [2, 2])
nn.Ws

[array([[ 0.75199925, -0.12880516],
        [ 0.24379904,  0.57679883],
        [-0.62916955, -0.95961575],
        [ 0.18466194,  0.55135709]]),
 array([[ 1.52069119, -0.05544439],
        [ 0.65529023, -1.79995309],
        [ 1.22749654, -1.28928801]]),
 array([[ 0.34015341],
        [-1.24326436],
        [-0.14630137]])]

In [21]:
x = [[1, 2, 3], [-1, 2, 3]]
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.11235281],
       [0.17501872]])

In [22]:
nn.mutate(scale=0.3)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.01043926],
       [0.16631624]])

In [23]:
nn.mutate(scale=0.1)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.00478018],
       [0.10258957]])

In [24]:
nn.mutate(scale=0.01)
y = nn.forward_pass(x, 'relu', 'sigmoid')
y

array([[0.00496172],
       [0.10198647]])